### 1. Importing Libraries

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator # for data augmentation
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


### 2. Data Preprocessing

In [2]:
num_classes = 5 # Happy, Sad, Angry, Neutral, Surprise
img_rows, img_cols = 48,48 # target img size
batch_size = 32

In [3]:
train_data_dir = "train/"
validation_data_dir = "validation/"

In [4]:
train_data_gen = ImageDataGenerator(
                    rescale=1./255, # rescaling img btw 0-1
                    rotation_range=30,
                    shear_range=0.3,
                    zoom_range=0.3,
                    width_shift_range=0.4,
                    height_shift_range=0.4,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [5]:
validation_data_gen = ImageDataGenerator(rescale=1./255) # For tsting model while training, so only 
                                                         # rescaling is required

In [6]:
train_generator = train_data_gen.flow_from_directory(
                        train_data_dir,
                        color_mode='grayscale', # imgs are gray
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
                        class_mode='categorical', # 5 classes
                        shuffle=True)

Found 24256 images belonging to 5 classes.


In [7]:
validation_generator = validation_data_gen.flow_from_directory(
                        validation_data_dir,
                        color_mode='grayscale', # imgs are gray
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
                        class_mode='categorical', # 5 classes
                        shuffle=True)

Found 3006 images belonging to 5 classes.


### 3. Model Architecture

In [8]:
model = Sequential()

In [9]:
# Block - 1

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
# Block - 2

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [11]:
# Block - 3

model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [12]:
# Block - 4

model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [13]:
# Block - 5

model.add(Flatten())
model.add(Dense(units=64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [14]:
# Block - 6

model.add(Dense(units=64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [15]:
# Block - 7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

### 4. Model Training

In [17]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [18]:
checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1)

In [19]:
earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=9,
                         verbose=1,
                         restore_best_weights=True)

In [20]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [21]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [22]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [23]:
nb_train_samples = 24256
nb_validation_samples = 3006
epochs = 25

In [24]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=nb_train_samples//batch_size,
                             epochs=epochs,
                             callbacks=callbacks,
                             validation_data=validation_generator,
                             validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
758/758 [==============================] - 1067s 1s/step - loss: 1.8533 - accuracy: 0.2428 - val_loss: 1.5016 - val_accuracy: 0.2930

Epoch 00001: val_loss improved from inf to 1.50159, saving model to Emotion_little_vgg.h5
Epoch 2/25
758/758 [==============================] - 711s 939ms/step - loss: 1.5776 - accuracy: 0.2837 - val_loss: 1.6782 - val_accuracy: 0.3137

Epoch 00002: val_loss did not improve from 1.50159
Epoch 3/25
758/758 [==============================] - 732s 965ms/step - loss: 1.5493 - accuracy: 0.3045 - val_loss: 1.6025 - val_accuracy: 0.2569

Epoch 00003: val_loss did not improve from 1.50159
Epoch 4/25
758/758 [==============================] - 694s 916ms/step - loss: 1.5317 - accuracy: 0.3132 - val_loss: 1.4111 - val_accuracy: 0.3362

Epoch 00004: val_loss improved from 1.50159 to 1.41108, saving model to Emotion_little_vgg.h5
Epoch 5/25
758/758 [==============================] - 729s 962ms/step - loss: 1.4704 - accuracy: 0.3509 - val_loss: 1.7782 - val